In [ ]:
import json
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score
from torch.optim import Adam

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from tqdm import tqdm_notebook as tqdm

from transformers import AutoTokenizer, AutoConfig, AutoModel

from crf_layer import CRFLayer
from multiLabelTokenClassfication import MultiLabelTokenClassification

from utils import read_by_lines, extract_result_multilabel

In [ ]:
device_Id = "2"

shema_path = './dictionary/event_schema.json'
enerm_dict_path = './dictionary/enum_tag.dict'
trigger_dict_path = './dictionary/trigger_tag.dict'
role_dict_path = './dictionary/role_tag.dict'

enerm_stacked_model_path = './models/DuEE_fin/stacked/stacked_enum.dict'
tigger_stacked_model_path = './models/DuEE_fin/stacked/stacked_trigger-multilabel.dict'
role_stacked_model_path = './models/DuEE_fin/stacked/stacked_role-multilabel-trick1.dict'

duee_fin_test_preprocess_path = './resources/duee_fin_test2_preprocess.json'

enum_role = "环节"
enum_event_type = "公司上市"
max_seq_len = 512

In [ ]:
def load_dict(dict_path):
    """load_dict"""
    vocab = {}
    for line in open(dict_path, 'r', encoding='utf-8'):
        value, key = line.strip('\n').split('\t')
        vocab[key] = int(value)
    return vocab

In [ ]:
label_enum_vocab = load_dict(dict_path=enerm_dict_path)
id2enumlabel = {val: key for key, val in label_enum_vocab.items()}
label_trigger_vocab = load_dict(dict_path=trigger_dict_path)
id2triggerlabel = {val: key for key, val in label_trigger_vocab.items()}
label_role_vocab = load_dict(dict_path=role_dict_path)
id2rolelabel = {val: key for key, val in label_role_vocab.items()}

In [ ]:
def enum_data_process(dataset):
    """enum_data_process"""
    output = []
    for d_json in dataset:
        text = d_json["text"].lower().replace("\t", " ")
        output.append({
            "id": d_json["id"],
            "sent_id": d_json["sent_id"],
            "text": text
        })
    return output

In [ ]:
def trigger_data_process(dataset):
    """data_process"""
    
    def replace_control_chars(str):
        if str == '\u200b' or str == '\ufeff' or str == '\ue601' or str == '\u3000':
            return '[UNK]'
        else:
            return str

    output = []
    for d_json in dataset:
        _id = d_json["id"]
        text_a = [
            "，" if t == " " or t == "\n" or t == "\t" else replace_control_chars(t)
            for t in list(d_json["text"].lower())
        ]
        output.append({
            "id": d_json["id"],
            "sent_id": d_json["sent_id"],
            "text": d_json["text"],
            "tokens": text_a
        })
    return output

In [ ]:
def role_data_process(dataset, trigger_file_path):
    """data_process"""
    
    def replace_control_chars(str):
        if str == '\u200b' or str == '\ufeff' or str == '\ue601' or str == '\u3000':
            return '[UNK]'
        else:
            return str
    
    trigger_data = read_by_lines(trigger_file_path)
    # process trigger data
    sent_trigger_mapping = {}
    for d in tqdm(trigger_data, total=len(trigger_data)):
        d_json = json.loads(d)
        t_ret = extract_result_multilabel(d_json["text"], d_json["pred"]["labels"])
        pred_event_types = list(set([(t["type"], ''.join(t["text"])) for t in t_ret]))
        if d_json["id"] not in sent_trigger_mapping:
            sent_trigger_mapping[d_json["id"]] = []
        for pred_event_type in pred_event_types:
            if pred_event_type not in sent_trigger_mapping[d_json["id"]]:
                sent_trigger_mapping[d_json["id"]].append(pred_event_type)

    output = []
    for d_json in dataset:
        _id = d_json["id"]
        text_a = [
            "，" if t == " " or t == "\n" or t == "\t" else replace_control_chars(t)
            for t in list(d_json["text"].lower())
        ]
        for pred_event_type in sent_trigger_mapping.get(d_json["id"], []):
            trigger_text = pred_event_type[0] + f"({pred_event_type[1]})："
            text_trigger = [
                "，" if t == " " or t == "\n" or t == "\t" else t
                for t in list(trigger_text.lower())
            ]
            output.append({
                "id": d_json["id"],
                "sent_id": d_json["sent_id"],
                "org_text": d_json["text"],
                "text": trigger_text + d_json["text"],
                "event_type": pred_event_type[0],
                "trigger": pred_event_type[1],
                "tokens": text_trigger+text_a
            })
    return output

In [ ]:
model_dict = {
    'ernie-base': AutoTokenizer.from_pretrained("nghuyong/ernie-1.0"),
    'roberta-chinese-base': AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext"),
    'roberta-chinese-large': AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext-large"),
}

In [ ]:
class BaiduEnermDataset(Dataset):
    
    def __init__(self, dataset_path):
        self.examples = []
        with open(dataset_path, 'r', encoding='utf-8') as f:
            dataset = json.loads(f.read())
            preprocess_dataset = enum_data_process(dataset)
            for d_json in tqdm(preprocess_dataset, total=len(preprocess_dataset)):
                text = d_json['text']
                b_input_ids, b_attention_masks, b_token_type_ids = [], [], []
                for tokenizer in model_dict.values():
                    PADDING = tokenizer.vocab[tokenizer.pad_token]
                    SEP = tokenizer.vocab[tokenizer.sep_token]
                    input_ids = tokenizer(text, is_split_into_words=False, add_special_tokens=True, max_length=max_seq_len, truncation=True)['input_ids']
                    tokens_input = input_ids + [PADDING] * (max_seq_len - len(input_ids))
                    attention_masks = self._get_attention_mask(input_ids, max_seq_len)
                    token_type_ids = self._get_token_type_id(input_ids, max_seq_len, sep_token=SEP)
                    b_input_ids.append(tokens_input)
                    b_attention_masks.append(attention_masks)
                    b_token_type_ids.append(token_type_ids)
                example = {
                    "input_ids": b_input_ids, "attention_masks": b_attention_masks, "token_type_ids": b_token_type_ids
                }
                example.update(d_json)
                self.examples.append(example)
                
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, item_idx):
        example = {
            "id": self.examples[item_idx]["id"],
            "sent_id": self.examples[item_idx]["sent_id"],
            "text": self.examples[item_idx]["text"],
            "input_ids": torch.tensor(self.examples[item_idx]["input_ids"]).long(),
            "attention_masks": torch.tensor(self.examples[item_idx]["attention_masks"]),
            "token_type_ids": torch.tensor(self.examples[item_idx]["token_type_ids"]),
        }
        return example

    def _get_attention_mask(self, input_ids, max_seq_len):
        """Mask for padding."""
        if len(input_ids) > max_seq_len:
            raise IndexError("Token length more than max seq length!")
        return [1] * len(input_ids) + [0] * (max_seq_len - len(input_ids))

    def _get_token_type_id(self, input_ids, max_seq_len, sep_token):
        """Segments: 0 for the first sequence, 1 for the second."""
        if len(input_ids) > max_seq_len:
            raise IndexError("Token length more than max seq length!")
        segments = []
        current_segment_id = 0
        for input_id in input_ids:
            segments.append(current_segment_id)
            if input_id == sep_token:
                current_segment_id = 1
        return segments + [0] * (max_seq_len - len(input_ids))

In [ ]:
class BaiduTriggerDataset(Dataset):
    
    def __init__(self, dataset_path):
        self.examples = []
        with open(dataset_path, 'r', encoding='utf-8') as f:
            dataset = json.loads(f.read())
            preprocess_dataset = trigger_data_process(dataset)
            for d_json in tqdm(preprocess_dataset, total=len(preprocess_dataset)):
                tokens = d_json['tokens']
                b_input_ids, b_attention_masks, b_token_type_ids = [], [], []
                for tokenizer in model_dict.values():
                    PADDING = tokenizer.vocab[tokenizer.pad_token]
                    SEP = tokenizer.vocab[tokenizer.sep_token]
                    input_ids = tokenizer(tokens, is_split_into_words=True, add_special_tokens=True, max_length=max_seq_len, truncation=True)['input_ids']
                    tokens_input = input_ids + [PADDING] * (max_seq_len - len(input_ids))
                    attention_masks = self._get_attention_mask(input_ids, max_seq_len)
                    token_type_ids = self._get_token_type_id(input_ids, max_seq_len, sep_token=SEP)
                    b_input_ids.append(tokens_input)
                    b_attention_masks.append(attention_masks)
                    b_token_type_ids.append(token_type_ids)
                example = {
                    "input_ids": b_input_ids, "attention_masks": b_attention_masks, "token_type_ids": b_token_type_ids, "seq_lens": len(tokens)
                }
                example.update(d_json)
                self.examples.append(example)

    def _get_attention_mask(self, input_ids, max_seq_len):
        """Mask for padding."""
        if len(input_ids) > max_seq_len:
            raise IndexError("Token length more than max seq length!")
        return [1] * len(input_ids) + [0] * (max_seq_len - len(input_ids))

    def _get_token_type_id(self, input_ids, max_seq_len, sep_token):
        """Segments: 0 for the first sequence, 1 for the second."""
        if len(input_ids) > max_seq_len:
            raise IndexError("Token length more than max seq length!")
        segments = []
        current_segment_id = 0
        for input_id in input_ids:
            segments.append(current_segment_id)
            if input_id == sep_token:
                current_segment_id = 1
        return segments + [0] * (max_seq_len - len(input_ids))       
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, item_idx):
        example = {
            "id": self.examples[item_idx]["id"],
            "sent_id": self.examples[item_idx]["sent_id"],
            "text": self.examples[item_idx]["text"],
            "input_ids": torch.tensor(self.examples[item_idx]["input_ids"]).long(),
            "attention_masks": torch.tensor(self.examples[item_idx]["attention_masks"]),
            "token_type_ids": torch.tensor(self.examples[item_idx]["token_type_ids"]),
            "seq_lens": self.examples[item_idx]["seq_lens"],
        }
        return example

In [ ]:
class BaiduRoleDataset(Dataset):
    
    def __init__(self, dataset_path, trigger_file_path):
        self.examples = []
        with open(dataset_path, 'r', encoding='utf-8') as f:
            dataset = json.loads(f.read())
            preprocess_dataset = role_data_process(dataset, trigger_file_path=trigger_file_path)
            for d_json in tqdm(preprocess_dataset, total=len(preprocess_dataset)):
                tokens = d_json['tokens']
                b_input_ids, b_attention_masks, b_token_type_ids = [], [], []
                for tokenizer in model_dict.values():
                    PADDING = tokenizer.vocab[tokenizer.pad_token]
                    SEP = tokenizer.vocab[tokenizer.sep_token]
                    input_ids = tokenizer(tokens, is_split_into_words=True, add_special_tokens=True, max_length=max_seq_len, truncation=True)['input_ids']
                    tokens_input = input_ids + [PADDING] * (max_seq_len - len(input_ids))
                    attention_masks = self._get_attention_mask(input_ids, max_seq_len)
                    token_type_ids = self._get_token_type_id(input_ids, max_seq_len, sep_token=SEP)
                    b_input_ids.append(tokens_input)
                    b_attention_masks.append(attention_masks)
                    b_token_type_ids.append(token_type_ids)
                example = {
                    "input_ids": b_input_ids, "attention_masks": b_attention_masks, "token_type_ids": b_token_type_ids, "seq_lens": len(tokens)
                }
                example.update(d_json)
                self.examples.append(example)

    def _get_attention_mask(self, input_ids, max_seq_len):
        """Mask for padding."""
        if len(input_ids) > max_seq_len:
            raise IndexError("Token length more than max seq length!")
        return [1] * len(input_ids) + [0] * (max_seq_len - len(input_ids))

    def _get_token_type_id(self, input_ids, max_seq_len, sep_token):
        """Segments: 0 for the first sequence, 1 for the second."""
        if len(input_ids) > max_seq_len:
            raise IndexError("Token length more than max seq length!")
        segments = []
        current_segment_id = 0
        for input_id in input_ids:
            segments.append(current_segment_id)
            if input_id == sep_token:
                current_segment_id = 1
        return segments + [0] * (max_seq_len - len(input_ids))        
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, item_idx):
        example = {
            "id": self.examples[item_idx]["id"],
            "sent_id": self.examples[item_idx]["sent_id"],
            "event_type": self.examples[item_idx]["event_type"],
            "trigger": self.examples[item_idx]["trigger"],
            "text": self.examples[item_idx]["text"],
            "input_ids": torch.tensor(self.examples[item_idx]["input_ids"]).long(),
            "attention_masks": torch.tensor(self.examples[item_idx]["attention_masks"]),
            "token_type_ids": torch.tensor(self.examples[item_idx]["token_type_ids"]),
            "seq_lens": self.examples[item_idx]["seq_lens"],
        }
        return example

In [ ]:
test_enerm_dataset = BaiduEnermDataset(dataset_path=duee_fin_test_preprocess_path)
test_trigger_dataset = BaiduTriggerDataset(dataset_path=duee_fin_test_preprocess_path)

In [ ]:
len(test_enerm_dataset)

In [ ]:
len(test_trigger_dataset)

In [ ]:
def set_seed(seed = 42):
    """Set the seed for generating random numbers on all GPUs.

    It's safe to call this function if CUDA is not available; in that case, it is silently ignored.

    Args:
        seed (int, optional): random numbers on all GPUs. Defaults to 42.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device(f'cuda:{device_Id}' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0))
    
    print('Memory Usage:')
    print('Allocated:', torch.cuda.memory_allocated(0)/1024**3, 'GB')
    print('Cached:   ', torch.cuda.memory_reserved(0)/1024**3, 'GB')
    
    print('CUDA Device Count:', n_gpu)
    
set_seed(seed=42)

In [ ]:
def write_by_lines(path, data):
    """write the data"""
    with open(path, "w") as outfile:
        [outfile.write(d + "\n") for d in data]

In [ ]:
@torch.no_grad()
def test_enerm(model, test_dataloader):
    model.eval()
    step = 0
    eval_acc = 0.0
    eval_f1 = 0.0
    eval_loss = 0.0
    results = []
    test_iterator = tqdm(test_dataloader)
    for batch in test_iterator:
        _, logits = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_masks'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )

        probs = torch.sigmoid(logits).data.cpu()
        probs_ids = (probs > 0.5).numpy()
        probs = probs.numpy()
        for id_, sent_id, text, label_probs, p_id in zip(batch['id'], batch['sent_id'], batch['text'], probs.tolist(), probs_ids.tolist()):
            true_indices = np.argwhere(p_id).flatten()
            labels = [id2enumlabel[true_index] for true_index in true_indices]
            results.append({"id": id_, "sent_id": sent_id, "text": text, "pred":{"probs": label_probs, "label": labels}})
    return results

In [ ]:
@torch.no_grad()
def test_trigger(model, test_dataloader):
    model.eval()
    step = 0
    eval_acc = 0.0
    eval_f1 = 0.0
    eval_loss = 0.0
    results = []
    test_iterator = tqdm(test_dataloader)
    for batch in test_iterator:
        _, logits = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_masks'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )

        probs = torch.sigmoid(logits).data.cpu()
        probs_ids = (probs > 0.1).numpy()
        probs = probs.numpy()
        for id_, sent_id, text, p_list, p_ids, seq_len in zip(batch['id'], batch['sent_id'], batch['text'], probs.tolist(), probs_ids.tolist(), batch['seq_lens']):
            prob_multi, label_multi = [], []
            for index, pid in enumerate(p_ids[1: seq_len - 1]):
                true_indices = np.argwhere(pid).flatten()
                prob_multi.append(p_list[index])
                label_multi.append([id2triggerlabel[true_index] for true_index in true_indices])
            results.append({"id": id_, "sent_id":sent_id, "text": text, "pred": {"probs": prob_multi, "labels": label_multi}})
    return results

In [ ]:
@torch.no_grad()
def test_role(model, test_dataloader):
    model.eval()
    step = 0
    eval_acc = 0.0
    eval_f1 = 0.0
    eval_loss = 0.0
    results = []
    test_iterator = tqdm(test_dataloader)
    for batch in test_iterator:
        _, logits = model(
            input_ids=batch['input_ids'].to(device),
            attention_mask=batch['attention_masks'].to(device),
            token_type_ids=batch['token_type_ids'].to(device)
        )

        probs = torch.sigmoid(logits).data.cpu()
        probs_ids = (probs > 0.1).numpy()
        probs = probs.numpy()
        for id_, sent_id, text, event_type, trigger, p_list, p_ids, seq_len in zip(batch['id'], batch['sent_id'], batch['text'], batch['event_type'], batch['trigger'], probs.tolist(), probs_ids.tolist(), batch['seq_lens']):
            prob_multi, label_multi = [], []
            for index, pid in enumerate(p_ids[1: seq_len - 1]):
                true_indices = np.argwhere(pid).flatten()
                prob_multi.append(p_list[index])
                label_multi.append([id2rolelabel[true_index] for true_index in true_indices])
            results.append({"id": id_, "sent_id":sent_id, "event_type": event_type, "trigger": trigger, "text": text, "pred": {"probs": prob_multi, "labels": label_multi}})
    return results

In [ ]:
test_enerm_sampler = SequentialSampler(test_enerm_dataset)
test_enerm_dataloader = DataLoader(test_enerm_dataset, sampler=test_enerm_sampler, batch_size = 64)

test_trigger_sampler = SequentialSampler(test_trigger_dataset)
test_trigger_dataloader = DataLoader(test_trigger_dataset, sampler=test_trigger_sampler, batch_size = 64)

In [ ]:
from stackedModel import MultiLabelStackedClassification

models = [
    torch.load(f'./models/DuEE_fin/{model_name}/enum.bin').to(device)
    for model_name in model_dict.keys()
]
enum_model = MultiLabelStackedClassification(models=models, num_labels=len(label_enum_vocab))
enum_model.load_state_dict(torch.load(enerm_stacked_model_path))

enum_model = enum_model.to(device)

models = [
    torch.load(f'./models/DuEE_fin/{model_name}/trigger-multilabel.bin').to(device)
    for model_name in model_dict.keys()
]
tigger_model = MultiLabelStackedClassification(models=models, num_labels=len(label_trigger_vocab))
tigger_model.load_state_dict(torch.load(tigger_stacked_model_path))

tigger_model = tigger_model.to(device)


models = [
    torch.load(f'./models/DuEE_fin/{model_name}/role-multilabel-trick1.bin').to(device)
    for model_name in model_dict.keys()
]
role_model = MultiLabelStackedClassification(models=models, num_labels=len(label_role_vocab))
role_model.load_state_dict(torch.load(role_stacked_model_path))

role_model = role_model.to(device)

In [ ]:
sentences_enerm = test_enerm(enum_model, test_enerm_dataloader)
sentences_enerm = [json.dumps(sent_enerm, ensure_ascii=False) for sent_enerm in sentences_enerm]
write_by_lines('./predict/DuEE_fin_test2/enerm/test_pred.json', sentences_enerm)

In [ ]:
sentences_tigger = test_trigger(tigger_model, test_trigger_dataloader)
sentences_tigger = [json.dumps(sent_tigger, ensure_ascii=False) for sent_tigger in sentences_tigger]
write_by_lines('./predict/DuEE_fin_test2/trigger/test_pred.json', sentences_tigger)

In [ ]:
test_role_dataset = BaiduRoleDataset(dataset_path=duee_fin_test_preprocess_path, trigger_file_path='./predict/DuEE_fin_test2/trigger/test_pred.json')

test_role_sampler = SequentialSampler(test_role_dataset)
test_role_dataloader = DataLoader(test_role_dataset, sampler=test_role_sampler, batch_size = 64)

In [ ]:
test_role_dataset[0]

In [ ]:
sentences_role = test_role(role_model, test_role_dataloader)
sentences_role = [json.dumps(sent_role, ensure_ascii=False) for sent_role in sentences_role]
write_by_lines('./predict/DuEE_fin_test2/role/test_pred.json', sentences_role)

In [ ]:
### multilabel classfication with header trick use 

!python duee_fin_postprocess_trick1.py --trigger_file ./predict/DuEE_fin_test2/trigger/test_pred.json \
    --role_file ./predict/DuEE_fin_test2/role/test_pred.json \
    --enum_file ./predict/DuEE_fin_test2/enerm/test_pred.json --schema_file ./dictionary/event_schema.json \
    --save_path ./submit/DuEE_fin_test2/test_duee_fin_erine_multilabel-trick1-fix.json --multilabel